In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [10]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     54   10    9  111.46  490  110.40  1.05  0.01  103.82  459  102.45   
10000    52   20   19  110.97  468  109.59  1.38  0.00  103.58  441  101.96   
15000    51   30   29  110.75  457  109.21  1.54  0.00  103.58  434  101.86   
20000    52   40   39  110.61  450  108.98  1.63  0.00  103.65  436  101.86   
25000    53   50   49  110.52  446  108.82  1.70  0.00  103.59  433  101.76   
30000    52   60   59  110.45  442  108.69  1.75  0.00  103.47  425  101.56   
35000    52   70   69  110.39  440  108.59  1.80  0.00  103.50  428  101.56   
40000    52   80   79  110.34  437  108.51  1.83  0.00  103.38  422  101.43   
45000    52   90   89  110.30  436  108.44  1.86  0.00  103.43  424  101.46   
50000    52  100   99  110.27  434  108.38  1.89  0.00  103.40  422  101.42   
55000    52  110  109  110.24  433  108.32  1.92  0.00  103.24  418  101.22   
60000    54  120  119  110.21  431  108.28  1.94  0.00  103.11  412  101.08   
65000    52  130  129  110.19  430  108.24  1.95  0.00  103.18  417  101.16   
70000    54  140  139  110.17  429  108.20  1.97  0.00  103.17  415  101.12   
75000    56  150  149  110.16  428  108.17  1.99  0.00  103.00  411  100.93   
80000    59  160  159  110.14  428  108.14  2.00  0.00  103.21  416  101.16   
85000    54  170  169  110.13  427  108.11  2.01  0.00  103.16  417  101.10   
90000    53  180  179  110.11  426  108.09  2.03  0.00  103.07  411  101.01   
95000    52  190  189  110.10  426  108.06  2.04  0.00  103.35  417  101.27   
100000   53  200  199  110.09  425  108.04  2.05  0.00  103.34  419  101.26   
105000   53  210  209  110.08  425  108.02  2.06  0.00  103.29  418  101.22   
110000   53  220  219  110.07  424  108.00  2.07  0.00  103.23  412  101.15   
115000   52  230  229  110.06  424  107.99  2.07  0.00  103.33  418  101.24   
120000   52  240  239  110.05  423  107.97  2.08  0.00  103.26  416  101.16   
125000   53  250  249  110.05  423  107.95  2.09  0.00  103.09  412  101.00   
130000   53  260  259  110.04  422  107.94  2.10  0.00  103.20  412  101.08   
135000   53  270  269  110.03  422  107.92  2.10  0.00  103.18  411  101.05   
140000   54  280  279  110.02  422  107.91  2.11  0.00  103.07  409  100.98   
145000   53  290  289  110.02  421  107.90  2.12  0.00  103.12  410  101.01   
150000   53  300  299  110.01  421  107.88  2.12  0.00  103.32  418  101.22   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   57  701  200  109.88  413  107.61  2.26  0.00  103.31  415  101.09   
355000   58  711  210  109.88  413  107.60  2.26  0.00  103.20  408  100.97   
360000   58  721  220  109.87  413  107.60  2.26  0.00  103.07  408  100.83   
365000   56  731  230  109.87  413  107.59  2.26  0.00  103.22  410  101.01   
370000   60  741  240  109.87  413  107.59  2.27  0.00  103.34  411  101.10   
375000   56  751  250  109.87  413  107.58  2.27  0.00  103.30  411  101.09   
380000   59  761  260  109.86  413  107.58  2.27  0.00  103.31  414  101.08   
385000   58  771  270  109.86  413  107.58  2.27  0.00  103.12  409  100.89   
390000   56  781  280  109.86  413  107.57  2.27  0.00  103.21  410  100.99   
395000   58  791  290  109.86  413  107.57  2.28  0.00  103.44  415  101.22   
400000   56  801  300  109.85  412  107.56  2.28  0.00  103.44  417  101.23   
405000   56  811  310  109.85  412  107.56  2.28  0.00  103.25  412  101.02   
410000   57  821  320  109.85  412  107.55  2.28  0.00  103.24  409  101.01   
415000   56  831  330  109.85  412  107.55  2.28  0.00  103.31  414  101.08   
420000   57  841  340  109.84  412  107.55  2.28  0.00  103.31  414  101.07   
425000   58  851  350  109.84  412  107.54  2.29  0.00  103.52  418  101.30   
430000   57  861  360  109.84  412  107.54  2.29  0.00  103.25  412  101.02   
435000   60  871  370  109.84  412  107.54  2.29  0.00  103.26  412  101.

0 R: 1.000 P: 0.390 carry room pockets pocket small strap nice perfect hold space
   1 R: 0.247 P: 0.104 sleeve protection inch inside padding zipper material price nice bit
     11 R: 0.055 P: 0.055 ; & chromebook pro perfectly air perfect color sleeve samsung
     12 R: 0.087 P: 0.087 air cover pro mac hard scratches protect protection nice feel
   2 R: 0.198 P: 0.073 months zipper broke smell quality cheap weeks 've time bought
     21 R: 0.063 P: 0.063 cover color keyboard love screen protector perfectly apple hard mac
     22 R: 0.062 P: 0.062 bottom top plastic part back piece corners cover cracked feet
   3 R: 0.165 P: 0.075 quality price bought $ ... amazon buy made reviews !
     31 R: 0.049 P: 0.049 ! love color perfect recommend perfectly absolutely ... cute buy
     32 R: 0.041 P: 0.041 received item ordered arrived shipping order return seller amazon time
